# Análisis Predictivo de Derrames Cerebrales (Stroke)

En este notebook, realizaremos un análisis completo del conjunto de datos de derrames cerebrales. El objetivo es preprocesar los datos, visualizarlos y construir un modelo de machine learning para predecir si un paciente podría sufrir un derrame cerebral.

## 1. Instalación de Librerías

In [1]:
!pip install pandas scikit-learn matplotlib seaborn

## 2. Carga y Exploración Inicial de Datos

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar los datos
# La ruta del archivo es relativa a la ubicación de este notebook
df = pd.read_csv('C:\Users\ttvga\OneDrive\Escritorio\vel\Clase de Fund DS\Clase 3 - 6')

# Mostrar las primeras filas del dataframe
print("Primeras 5 filas del dataset:")
print(df.head())

# Mostrar información general del dataset
print("Información general del dataset:")
df.info()

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3847015651.py, line 8)

## 3. Limpieza y Preprocesamiento de Datos

### 3.1. Manejo de Valores Faltantes
Notamos que la columna `bmi` tiene valores faltantes ('N/A'). Los reemplazaremos con la media de la columna.

In [ ]:
# Reemplazar 'N/A' con NaN de numpy para que sea tratable numéricamente
df['bmi'].replace('N/A', np.nan, inplace=True)

# Convertir la columna bmi a tipo numérico
df['bmi'] = pd.to_numeric(df['bmi'])

# Calcular la media de la columna bmi
mean_bmi = df['bmi'].mean()

# Rellenar los valores NaN con la media
df['bmi'].fillna(mean_bmi, inplace=True)

print(f"Valores faltantes en 'bmi' rellenados con la media: {mean_bmi:.2f}")
print("Verificación de valores nulos después de la limpieza:")
print(df.isnull().sum())

### 3.2. Manejo de Datos Categóricos
Convertiremos las variables categóricas (como 'gender') en variables numéricas usando One-Hot Encoding para que el modelo pueda procesarlas.

In [ ]:
# Eliminar la fila con 'Other' en género, ya que es una sola y puede complicar el modelo sin aportar mucho
df = df[df['gender'] != 'Other']

# Aplicar One-Hot Encoding a las variables categóricas
df_processed = pd.get_dummies(df, drop_first=True)

print("Columnas después del One-Hot Encoding:")
print(df_processed.columns)

print("Primeras 5 filas del dataset procesado:")
print(df_processed.head())

## 4. Visualización de Datos (Gráficos)

In [ ]:
sns.set_style('whitegrid')

# Gráfico 1: Distribución de la variable objetivo 'stroke'
plt.figure(figsize=(6, 4))
sns.countplot(x='stroke', data=df)
plt.title('Distribución de Casos de Derrame Cerebral')
plt.show()

# Gráfico 2: Distribución de la edad
plt.figure(figsize=(10, 6))
sns.histplot(df['age'], kde=True, bins=30)
plt.title('Distribución de la Edad')
plt.show()

# Gráfico 3: Correlación entre variables numéricas
plt.figure(figsize=(12, 8))
sns.heatmap(df_processed.corr(), annot=False, cmap='coolwarm')
plt.title('Mapa de Calor de Correlaciones')
plt.show()

## 5. Construcción del Modelo Predictivo

### 5.1. Preparación de Datos para el Modelo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separar las características (X) y la variable objetivo (y)
X = df_processed.drop('stroke', axis=1)
y = df_processed['stroke']

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Escalar las características numéricas
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Datos de entrenamiento: {X_train.shape[0]} filas")
print(f"Datos de prueba: {X_test.shape[0]} filas")

### 5.2. Entrenamiento y Evaluación del Modelo (Regresión Logística)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Crear y entrenar el modelo
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Realizar predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Precisión (Accuracy) del modelo: {accuracy:.4f}")
print("Matriz de Confusión:")
print(conf_matrix)
print("Reporte de Clasificación:")
print(class_report)

### 5.3. Conclusión del Modelo
El reporte de clasificación nos muestra que, aunque la precisión general es alta (alrededor del 95%), esto se debe a que el modelo predice muy bien los casos de 'no derrame' (clase 0), que son la gran mayoría. Sin embargo, el rendimiento para predecir los casos de 'sí derrame' (clase 1) es muy bajo (bajo 'recall'). Esto es común en datasets desbalanceados y sugiere que se necesitarían técnicas más avanzadas (como el sobremuestreo de la clase minoritaria con SMOTE) para obtener un modelo más útil en la práctica.